In [1]:
import sqlalchemy
con = sqlalchemy.create_engine('postgres://localhost/baseball')


In [16]:
view = '''
create view choices as
select 
    game_id,
    bat_id,
    BAT_HOME_ID,
    BAT_LINEUP_ID,
    array_agg(BAT_HAND_CD),
    count(event_id), 
    max(h_cd) as best_hit, 
    count(case h_cd when '0' then null else 1 end) as num_hits, 
    count(case h_cd when '1' then 1 else null end) as num_singles,
    count(case h_cd when '2' then 1 else null end) as num_doubles, 
    count(case h_cd when '3' then 1 else null end) as num_triples,
    count(case h_cd when '4' then 1 else null end) as num_hrs 
    from vw_events 
        inner join vw_games using (game_id) 
    where 
        bat_start_fl='T' 
        and BAT_EVENT_FL='T'
        and bat_fld_cd != '1'
    group by game_id, bat_id, BAT_HOME_ID, BAT_LINEUP_ID
'''
con.execute("drop view if exists choices cascade;")
con.execute(view)
pd.read_sql("select * from choices", con)
# should be: 41278


,game_id,bat_id,bat_home_id,bat_lineup_id,array_agg,count,best_hit,num_hits,num_singles,num_doubles,num_triples,num_hrs
0,ANA201504100,aybae001,1,6,"[R, L, R, L]",4,0,0,0,0,0,0
1,ANA201504100,cainl001,0,3,"[R, R, R, R, R]",5,2,2,1,1,0,0
2,ANA201504100,calhk001,1,1,"[L, L, L, L]",4,2,1,0,1,0,0
3,ANA201504100,cowgc001,1,8,"[R, R]",2,1,1,1,0,0,0
4,ANA201504100,cronc002,1,5,"[R, R, R, R]",4,0,0,0,0,0,0
5,ANA201504100,escoa003,0,1,"[R, R, R, R, R]",5,1,1,1,0,0,0
6,ANA201504100,freed001,1,4,"[R, R, R, R]",4,4,2,1,0,0,1
7,ANA201504100,giavj001,1,9,"[R, R, R]",3,0,0,0,0,0,0
8,ANA201504100,gorda001,0,6,"[L, L, L, L]",4,0,0,0,0,0,0
9,ANA201504100,hosme001,0,4,"[L, L, L, L, L]",5,1,1,1,0,0,0


In [2]:
import pandas as pd
sql = '''
select distinct on (game_id, choices.bat_id)
    game_date,
    batter.first_name_tx,
    batter.last_name_tx,
    choices.*,
    BAT_HOME_ID as batter_home,
    case BAT_HOME_ID when '0' then HOME_START_PIT_ID else AWAY_START_PIT_ID end as opposing_pitcher_id,
    case BAT_HOME_ID when '1' then HOME_TEAM_ID else AWAY_TEAM_ID end as team_id,
    case BAT_HOME_ID when '0' then HOME_TEAM_ID else AWAY_TEAM_ID end as opposing_team_id,
    DH_FL,
    DAYNIGHT_PARK_CD,
    PARK_ID,
    AWAY_START_PIT_ID,
    HOME_START_PIT_ID,
    ATTEND_PARK_CT,
    TEMP_PARK_CT,
    WIND_SPEED_PARK_CT,
    FIELD_PARK_CD,
    PRECIP_PARK_CD,
    SKY_PARK_CD,
    batter.bat_hand_cd as bat_hand,
    opposing_pitcher.pit_hand_cd
from choices 
    inner join vw_games using (game_id)
    inner join rosters as batter on (choices.bat_id = batter.player_id)
    inner join rosters as opposing_pitcher on ((case BAT_HOME_ID when '0' then HOME_START_PIT_ID else AWAY_START_PIT_ID end) = opposing_pitcher.player_id)
'''

choices = pd.read_sql(sql, con)
choices

,game_date,first_name_tx,last_name_tx,game_id,bat_id,bat_home_id,bat_lineup_id,array_agg,count,best_hit,...,away_start_pit_id,home_start_pit_id,attend_park_ct,temp_park_ct,wind_speed_park_ct,field_park_cd,precip_park_cd,sky_park_cd,bat_hand,pit_hand_cd
0,2015-04-10,Erick,Aybar,ANA201504100,aybae001,1,6,"[R, L, R, L]",4,0,...,vargj001,santh001,44489,67,7,0,0,2,B,L
1,2015-04-10,Lorenzo,Cain,ANA201504100,cainl001,0,3,"[R, R, R, R, R]",5,2,...,vargj001,santh001,44489,67,7,0,0,2,R,L
2,2015-04-10,Kole,Calhoun,ANA201504100,calhk001,1,1,"[L, L, L, L]",4,2,...,vargj001,santh001,44489,67,7,0,0,2,L,L
3,2015-04-10,Collin,Cowgill,ANA201504100,cowgc001,1,8,"[R, R]",2,1,...,vargj001,santh001,44489,67,7,0,0,2,R,L
4,2015-04-10,C.J.,Cron,ANA201504100,cronc002,1,5,"[R, R, R, R]",4,0,...,vargj001,santh001,44489,67,7,0,0,2,R,L
5,2015-04-10,Alcides,Escobar,ANA201504100,escoa003,0,1,"[R, R, R, R, R]",5,1,...,vargj001,santh001,44489,67,7,0,0,2,R,L
6,2015-04-10,David,Freese,ANA201504100,freed001,1,4,"[R, R, R, R]",4,4,...,vargj001,santh001,44489,67,7,0,0,2,R,L
7,2015-04-10,Johnny,Giavotella,ANA201504100,giavj001,1,9,"[R, R, R]",3,0,...,vargj001,santh001,44489,67,7,0,0,2,R,L
8,2015-04-10,Alex,Gordon,ANA201504100,gorda001,0,6,"[L, L, L, L]",4,0,...,vargj001,santh001,44489,67,7,0,0,2,L,L
9,2015-04-10,Eric,Hosmer,ANA201504100,hosme001,0,4,"[L, L, L, L, L]",5,1,...,vargj001,santh001,44489,67,7,0,0,2,L,L


In [9]:
def calculate_previous_batter_games(frame):
    game_date = frame['game_date']
    player_id = frame['bat_id']
    
    sql = '''
    select event_cd,count(*) from vw_events 
        where BAT_EVENT_FL='T' and bat_id='{}' and game_date < '{}' 
        group by event_cd
    '''.format(player_id, game_date)
    
    games = pd.read_sql(sql, con)
    return games.set_index(games.event_cd)['count'].add_prefix('previous_batter_')

def calculate_previous_pitcher_games(frame):
    game_date = frame['game_date']
    player_id = frame['opposing_pitcher_id']
    
    sql = '''
    select event_cd,count(*) from vw_events 
        where BAT_EVENT_FL='T' and pit_id='{}' and game_date < '{}' 
        group by event_cd
    '''.format(player_id, game_date)
    
    games = pd.read_sql(sql, con)
    return games.set_index(games.event_cd)['count'].add_prefix('previous_pitcher_')

def calculate_pitcher_against_hand(frame):
    bat_hand_cd = frame['bat_hand']
    game_date = frame['game_date']
    player_id = frame['opposing_pitcher_id']

    sql = '''
    select event_cd,count(*) from vw_events 
        inner join rosters on (rosters.bat_hand_cd = '{}' and rosters.player_id = vw_events.bat_id)
        where BAT_EVENT_FL='T' and pit_id='{}' and game_date < '{}'
        group by event_cd

    '''.format(bat_hand_cd, player_id, game_date)

    games = pd.read_sql(sql, con)
    return games.set_index(games.event_cd)['count'].add_prefix('pitcher_against_hand_')
    
def calculate_batter_against_hand(frame):
    pit_hand_cd = frame['pit_hand_cd']
    game_date = frame['game_date']
    player_id = frame['bat_id']

    sql = '''
    select event_cd,count(*) from vw_events 
        where BAT_EVENT_FL='T' and pit_hand_cd='{}' and bat_id='{}' and game_date < '{}' 
        group by event_cd
    '''.format(pit_hand_cd, player_id, game_date)

    games = pd.read_sql(sql, con)
    return games.set_index(games.event_cd)['count'].add_prefix('batter_against_hand_')

bull_cache = dict()
def calculate_bullpen(frame):
    pit_team = frame['opposing_team_id']
    game_date = frame['game_date']
    
    if(bull_cache.get((pit_team, game_date), None) is not None):
        return bull_cache.get((pit_team, game_date))
    
    sql = '''
    select event_cd,count(*) from vw_events 
        where BAT_EVENT_FL='T' and game_date < '{}' and fld_team_id='{}' and PIT_START_FL='F'
        group by event_cd
    '''.format(game_date, pit_team)

    games = pd.read_sql(sql, con)
    result = games.set_index(games.event_cd)['count'].add_prefix('bullpen_')
    bull_cache[(pit_team, game_date)] = result
    
    return result

In [6]:
choices = choices.join(choices.apply(calculate_previous_batter_games, axis=1))
choices.to_csv('step1.csv')
choices = choices.join(choices.apply(calculate_previous_pitcher_games, axis=1))
choices.to_csv('step2.csv')

choices = choices.join(choices.apply(calculate_pitcher_against_hand, axis=1))
choices.to_csv('step3.csv')

choices = choices.join(choices.apply(calculate_batter_against_hand, axis=1))
choices.to_csv('step4.csv')

choices = choices.join(choices.apply(calculate_bullpen, axis=1))
choices.to_csv('step5.csv')

choices

,game_date,first_name_tx,last_name_tx,game_id,bat_id,bat_home_id,bat_lineup_id,array_agg,count,best_hit,...,batter_against_hand_16,batter_against_hand_17,batter_against_hand_18,batter_against_hand_19,batter_against_hand_2,batter_against_hand_20,batter_against_hand_21,batter_against_hand_22,batter_against_hand_23,batter_against_hand_3
0,2015-04-10,Erick,Aybar,ANA201504100,aybae001,1,6,"[R, L, R, L]",4,0,...,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN
1,2015-04-10,Lorenzo,Cain,ANA201504100,cainl001,0,3,"[R, R, R, R, R]",5,2,...,1.0,NaN,NaN,NaN,4.0,1.0,1.0,NaN,NaN,NaN
2,2015-04-10,Kole,Calhoun,ANA201504100,calhk001,1,1,"[L, L, L, L]",4,2,...,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,2.0
3,2015-04-10,Collin,Cowgill,ANA201504100,cowgc001,1,8,"[R, R]",2,1,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,2015-04-10,C.J.,Cron,ANA201504100,cronc002,1,5,"[R, R, R, R]",4,0,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0
5,2015-04-10,Alcides,Escobar,ANA201504100,escoa003,0,1,"[R, R, R, R, R]",5,1,...,NaN,NaN,NaN,NaN,4.0,1.0,1.0,NaN,NaN,NaN
6,2015-04-10,David,Freese,ANA201504100,freed001,1,4,"[R, R, R, R]",4,4,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
7,2015-04-10,Johnny,Giavotella,ANA201504100,giavj001,1,9,"[R, R, R]",3,0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
8,2015-04-10,Alex,Gordon,ANA201504100,gorda001,0,6,"[L, L, L, L]",4,0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0
9,2015-04-10,Eric,Hosmer,ANA201504100,hosme001,0,4,"[L, L, L, L, L]",5,1,...,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,1.0,1.0


In [15]:
len(bull_cache)


4800